In [1]:
%pip install -q -U keras-tuner
%pip install -q -U tensorboard-plugin-profile

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.2.1 -> 23.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.2.1 -> 23.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
import pandas as pd
import tensorflow as tf
from tensorflow import keras
import keras_tuner as kt

## Data Fetching

In [3]:
from utils.data_fetch import read_data
data = read_data()

C:\Users\Vardan\AppData\Local\Temp\ipykernel_9848\1164869249.py:2: DeprecationWarning: [Deprecated][in version 6.0.0]: client_factory will be replaced by gspread.http_client types
  data = read_data()


In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 27 entries, 0 to 26
Data columns (total 5 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Spindle Speed     27 non-null     float64
 1   Feed (mm/min)     27 non-null     float64
 2   Tool Dia. (mm)    27 non-null     float64
 3   MRR (mm^3 / min)  27 non-null     float64
 4   L_e               27 non-null     float64
dtypes: float64(5)
memory usage: 1.2 KB


## Data Preprocessing

In [5]:
from sklearn.model_selection import train_test_split
train_set, test_set = train_test_split(data, test_size=0.2, random_state=32)

In [6]:
X_train_full = train_set.drop(["L_e", "MRR (mm^3 / min)"], axis = 1)
y_train_full =pd.concat([train_set["L_e"], train_set["MRR (mm^3 / min)"]], axis=1)

In [7]:
train_set, val_set = train_test_split(train_set, test_size=0.2, random_state=32)

In [8]:
X_train = train_set.drop(["L_e", "MRR (mm^3 / min)"], axis = 1)
y_train=pd.concat([train_set["L_e"], train_set["MRR (mm^3 / min)"]], axis=1)

In [9]:
X_val = val_set.drop(["L_e", "MRR (mm^3 / min)"], axis = 1)
y_val=pd.concat([val_set["L_e"], val_set["MRR (mm^3 / min)"]], axis=1)

In [10]:
X_test = test_set.drop(["L_e", "MRR (mm^3 / min)"], axis = 1)
y_test=pd.concat([test_set["L_e"], test_set["MRR (mm^3 / min)"]], axis=1)

In [11]:
X_train.shape

(16, 3)

In [12]:
norm_layer = keras.layers.Normalization(input_shape=X_train.shape[1:])

In [13]:
model = keras.models.Sequential([
    norm_layer,
    keras.layers.BatchNormalization(),
    keras.layers.Dense(500, activation="relu", kernel_initializer="he_normal"),
    keras.layers.BatchNormalization(),
    keras.layers.Dense(300, activation="relu", kernel_initializer="he_normal"),
    keras.layers.BatchNormalization(),
    keras.layers.Dense(100, activation="relu", kernel_initializer="he_normal"),
    keras.layers.Dense(2)
])
model.compile(loss="mse", optimizer=keras.optimizers.Adam(learning_rate=1e-3), metrics=["RootMeanSquaredError", "mse"])
norm_layer.adapt(X_train)

In [14]:
X_train.shape

(16, 3)

In [15]:
y_train.shape

(16, 2)

In [16]:
X_val.shape

(5, 3)

In [17]:
y_val.shape

(5, 2)

In [18]:
from pathlib import Path
from time import strftime
def get_run_logdir(root_logdir="my_logs"):
 return Path(root_logdir) / strftime("run_%Y_%m_%d_%H_%M_%S")
run_logdir = get_run_logdir() 

In [19]:
early_stopping_cb = tf.keras.callbacks.EarlyStopping(patience=20, restore_best_weights=True)

In [20]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 normalization (Normalizati  (None, 3)                 7         
 on)                                                             
                                                                 
 batch_normalization (Batch  (None, 3)                 12        
 Normalization)                                                  
                                                                 
 dense (Dense)               (None, 500)               2000      
                                                                 
 batch_normalization_1 (Bat  (None, 500)               2000      
 chNormalization)                                                
                                                                 
 dense_1 (Dense)             (None, 300)               150300    
                                                        

In [21]:
history = model.fit(X_train,y_train, epochs=500, verbose=1, callbacks=[ early_stopping_cb],validation_data=(X_val,y_val))

Epoch 1/500


1/1 [==============================] - 3s 3s/step - loss: 31.1460 - root_mean_squared_error: 5.5809 - mse: 31.1460 - val_loss: 45.9055 - val_root_mean_squared_error: 6.7754 - val_mse: 45.9055
Epoch 2/500
1/1 [==============================] - 0s 116ms/step - loss: 16.5761 - root_mean_squared_error: 4.0714 - mse: 16.5761 - val_loss: 42.2409 - val_root_mean_squared_error: 6.4993 - val_mse: 42.2409
Epoch 3/500
1/1 [==============================] - 0s 69ms/step - loss: 10.1891 - root_mean_squared_error: 3.1920 - mse: 10.1891 - val_loss: 39.0023 - val_root_mean_squared_error: 6.2452 - val_mse: 39.0023
Epoch 4/500
1/1 [==============================] - 0s 53ms/step - loss: 6.7057 - root_mean_squared_error: 2.5895 - mse: 6.7057 - val_loss: 36.2376 - val_root_mean_squared_error: 6.0198 - val_mse: 36.2376
Epoch 5/500
1/1 [==============================] - 0s 63ms/step - loss: 4.2677 - root_mean_squared_error: 2.0658 - mse: 4.2677 - val_loss: 33.7874 - val_root_mean_squared_error:

KeyboardInterrupt: 

In [ ]:
rmse_score = model.evaluate(X_test,y_test)

In [ ]:
X_new = X_train[:3]

In [ ]:
X_train[:3]

In [ ]:
model.predict(X_new)

In [ ]:
y_train[:3]

### Model Tuning

In [ ]:
def build_model(hp):
    n_hidden = hp.Int("n_hidden", min_value=0, max_value=8, default=2)
    n_neurons = hp.Int("n_neurons", min_value=100, max_value=500)
    learning_rate = hp.Float("learning_rate", min_value=1e-4, max_value=1e-2,sampling="log")
    optimizer = keras.optimizers.Adam(learning_rate=learning_rate)
    model = keras.Sequential()
    model.add(norm_layer)
    for _ in range(n_hidden):
        model.add(keras.layers.BatchNormalization())
        model.add(keras.layers.Dense(n_neurons, activation="relu", kernel_initializer="he_normal"))
    model.add(keras.layers.Dense(2))
    model.compile(loss="mse", optimizer=optimizer, metrics=["RootMeanSquaredError"])
    return model

In [ ]:
tuner = kt.RandomSearch(
 build_model, objective=kt.Objective("val_root_mean_squared_error", direction="min"), max_trials=10, overwrite=True,
 directory="Model_Tuning", project_name="my_rnd_search", seed=42)

early_stop = keras.callbacks.EarlyStopping(monitor='val_loss', patience=30)

tuner.search(X_train, y_train, epochs=150, validation_data=(X_val, y_val), callbacks=[early_stop])

In [ ]:
best_hps = tuner.get_best_hyperparameters(num_trials=1)[0]

In [ ]:
best_trial = tuner.oracle.get_best_trials(num_trials=1)[0]

In [ ]:
best_trial.summary()

In [ ]:
Final_model = tuner.get_best_models(num_models=1)[0]

In [ ]:
tensorboard_cb = tf.keras.callbacks.TensorBoard(run_logdir, profile_batch=(100, 200))

In [ ]:
early_stopping_cb = tf.keras.callbacks.EarlyStopping(patience=10, restore_best_weights=True)

In [ ]:
Final_model.fit(X_train_full, y_train_full, epochs=200, callbacks=[tensorboard_cb, early_stopping_cb])

In [ ]:
%load_ext tensorboard
%tensorboard --logdir=./my_logs

In [ ]:
X_new

In [ ]:
Final_model.predict(X_new)

In [ ]:
y_train[:3]

In [ ]:
Final_model.save("Final_neural_network.h5")

In [22]:
model = tf.keras.models.load_model('Final_neural_network.h5')

In [26]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 normalization_1 (Normaliza  (None, 3)                 7         
 tion)                                                           
                                                                 
 batch_normalization (Batch  (None, 3)                 12        
 Normalization)                                                  
                                                                 
 dense (Dense)               (None, 456)               1824      
                                                                 
 batch_normalization_1 (Bat  (None, 456)               1824      
 chNormalization)                                                
                                                                 
 dense_1 (Dense)             (None, 456)               208392    
                                                        

In [24]:
model.evaluate(X_train, y_train)

1/1 [==============================] - 0s 251ms/step - loss: 0.0925 - root_mean_squared_error: 0.3041


[0.09247415512800217, 0.30409565567970276]

In [25]:
model.evaluate(X_test,y_test)

1/1 [==============================] - 0s 38ms/step - loss: 1.2252 - root_mean_squared_error: 1.1069


[1.2251598834991455, 1.1068694591522217]

In [ ]:
y_pred = model.predict(X_test)

In [ ]:
y_pred

In [ ]:
y_test

In [ ]:
! pip install ann_visualizer